In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 25.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.5 MB/s eta 0:00:00


In [2]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00


In [3]:
import tensorflow as tf
import numpy
import matplotlib.pyplot as plt
import sklearn
import cv2
from sklearn.metrics import confusion_matrix,roc_curve
import seaborn as sns
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryFocalCrossentropy,categorical_crossentropy,sparse_categorical_crossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy,CategoricalAccuracy,SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import (BertTokenizerFast,TFBertTokenizer,BertTokenizer,RobertaTokenizerFast,DataCollatorWithPadding,TFRobertaForSequenceClassification,TFBertForSequenceClassification,TFBertModel,create_optimizer)



In [4]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [5]:
BATCH_SIZE=16

In [6]:
dataset_id="imdb"
dataset=load_dataset(dataset_id)

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [7]:
model_id="microsoft/xtremedistil-l6-h256-uncased"
tokenizer=BertTokenizerFast.from_pretrained(model_id)

In [8]:
def preprocess_function(examples):
  return tokenizer(examples['text'],max_length=512,padding=True,truncation=True)

In [9]:
tokenized_dataset=dataset.map(preprocess_function,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [10]:
tf_train_dataset=tokenized_dataset["train"].to_tf_dataset(
    columns=['input_ids','token_type_ids','attention_mask','label'],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

In [11]:
tf_val_dataset=tokenized_dataset["test"].to_tf_dataset(
    columns=['input_ids','token_type_ids','attention_mask','label'],
    shuffle=True,
    batch_size=BATCH_SIZE,
)

In [12]:
def swap_positions(dataset):
  return{'input_ids':dataset['input_ids'],
         'token_type_ids':dataset['token_type_ids'],
         'attention_mask':dataset['attention_mask'],},dataset['label']

In [13]:
tf_train_dataset=tf_train_dataset.map(swap_positions).prefetch(tf.data.AUTOTUNE)
tf_val_dataset=tf_val_dataset.map(swap_positions).prefetch(tf.data.AUTOTUNE)

In [14]:
for i in tf_val_dataset.take(1):
  print(i)

({'input_ids': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[  101,  2007,  1037, ...,     0,     0,     0],
       [  101,  2045,  2024, ...,     0,     0,     0],
       [  101,  1045,  2293, ...,     0,     0,     0],
       ...,
       [  101,  1045,  1005, ...,     0,     0,     0],
       [  101, 14732,  9350, ...,     0,     0,     0],
       [  101,  1012,  1012, ...,  8462, 12991,   102]])>, 'token_type_ids': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])>, 'attention_mask': <tf.Tensor: shape=(16, 512), dtype=int64, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1]])>}, <tf.Tensor: shape=(16,), dtype=int64

In [15]:
model=TFBertForSequenceClassification.from_pretrained(model_id,num_labels=2)
model.summary()

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at microsoft/xtremedistil-l6-h256-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  12750080  
                                                                 
 dropout_19 (Dropout)        multiple                  0         
                                                                 
 classifier (Dense)          multiple                  514       
                                                                 
Total params: 12750594 (48.64 MB)
Trainable params: 12750594 (48.64 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
num_epochs=3
batches_per_epoch=len(tokenized_dataset["train"])//BATCH_SIZE
total_train_steps=int(batches_per_epoch*num_epochs)

In [17]:
optimizer,schedule=create_optimizer(init_lr=2e-5,num_warmup_steps=0,num_train_steps=total_train_steps)


In [18]:
checkpoint_filepath="/content/sample_data/modelcheckpoint"
model_checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [19]:
model.compile(
    optimizer=optimizer,
    metrics=['accuracy'],)


In [20]:
history=model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=5,
)

Epoch 1/5
1563/1563 [==============================] - 704s 430ms/step - loss: 0.3036 - accuracy: 0.8761 - val_loss: 0.2397 - val_accuracy: 0.9066
Epoch 2/5
1563/1563 [==============================] - 663s 424ms/step - loss: 0.2163 - accuracy: 0.9159 - val_loss: 0.2083 - val_accuracy: 0.9190
Epoch 3/5
1563/1563 [==============================] - 657s 421ms/step - loss: 0.1880 - accuracy: 0.9274 - val_loss: 0.2112 - val_accuracy: 0.9204
Epoch 4/5
1563/1563 [==============================] - 618s 396ms/step - loss: 0.1776 - accuracy: 0.9326 - val_loss: 0.2112 - val_accuracy: 0.9204
Epoch 5/5
1563/1563 [==============================] - 616s 394ms/step - loss: 0.1783 - accuracy: 0.9315 - val_loss: 0.2112 - val_accuracy: 0.9204


In [21]:
 model.save("/content/sample_data/distil")

In [22]:
input=tokenizer(["Baahubali is one of the most visually impressive films that have come from India."],padding=True,return_tensors='tf')
logits=model(**input).logits
print(logits)

tf.Tensor([[-2.617712  2.278574]], shape=(1, 2), dtype=float32)
